In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 1. Define the traditional MLP architecture
def build_traditional_mlp(input_shape):
    model = models.Sequential([
        # Input layer: Must match the number of features in your data
        layers.Input(shape=(input_shape,)),
        
        # Hidden Layer 1: Learn patterns using the ReLU activation function
        layers.Dense(64, activation='relu'), 
        
        # Hidden Layer 2: A smaller layer to refine the patterns
        layers.Dense(16, activation='relu'),
        
        # Output Layer: Single neuron with Sigmoid for probability (0 to 1)
        layers.Dense(1, activation='sigmoid')
    ])
    
    # 2. Compile the model
    model.compile(
        optimizer='adam', # Standard "weight updater"
        loss='binary_crossentropy', # Measures how "wrong" the model is
        metrics=['accuracy']
    )
    return model